In [1]:
# Copyright 2025 Forusone
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

## Agent Engine in Vertex AI - Multiple tools




### Install Vertex AI SDK for Python

In [2]:
%pip install --upgrade --user --quiet google-cloud-aiplatform[agent_engines,adk,langchain,ag2,llama_index]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 23.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 94.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.5/229.5 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.7/153.7 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 734.2/734.2 kB 34.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.1/232.1 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.5/215.5 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.1/334.1 kB 22.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [3]:
%pip install --upgrade --user --quiet google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.7/159.7 kB 6.9 MB/s eta 0:00:00


In [1]:
# Note:  Need to restart the kernel
%pip install --upgrade --user --quiet langchain-google-vertexai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 1.3 MB/s eta 0:00:00


In [2]:
exit()

In [ ]:
# @title Authentication to access to GCP

# To use markdown for output data from LLM
from IPython.display import display, Markdown

# @title Define constants
PROJECT_ID = "ai-hangsik"
LOCATION = "us-central1"
MODEL_NAME = "gemini-2.0-flash"
BUCKET_URI = f"gs://agent-0417"

# Use OAuth to access the GCP environment.
import sys
if "google.colab" in sys.modules:
    from google.colab import auth
    auth.authenticate_user(project_id = PROJECT_ID)

In [ ]:
# @title Create a bucket.
! gsutil mb -l {LOCATION} -p {PROJECT_ID} {BUCKET_URI}

In [ ]:
# @title Initialize Vertex AI with Staging Bucket.

import vertexai

vertexai.init(project=PROJECT_ID, location=LOCATION, staging_bucket=BUCKET_URI)

### Tools definition

`Note : Unsupported tool call type function_declarations=None retrieval=None google_search=GoogleSearch() google_search_retrieval=None code_execution=None `

In [ ]:
#@title 1. Use function calling for a tool.

def get_exchange_rate(
    currency_from: str = "USD",
    currency_to: str = "KRW",
    currency_date: str = "latest",
):
    """Retrieves the exchange rate between two currencies on a specified date."""

    import requests

    print(f"Retrieving exchange rate from {currency_from} to {currency_to} on {currency_date}")

    response = requests.get(
        f"https://api.frankfurter.app/{currency_date}",
        params={"from": currency_from, "to": currency_to},
    )

    print(response)

    return response.json()

In [ ]:
#@title 2. Use LLM model to make a tool

from google.genai import types
from google import genai

def get_capital_return(
    country:str, ):

  """Retrieves the capital city name of the given country"""

  from google import genai
  from google.genai.types import HttpOptions
  from google.genai.types import Tool, GenerateContentConfig, GoogleSearch

  prompt = f"Answer the capital city of {country}"

  client = genai.Client(vertexai=True,
                        project=PROJECT_ID,
                        location=LOCATION,
                        http_options=HttpOptions(api_version="v1"))

  google_search_tool = Tool(
      google_search = GoogleSearch()
  )

  # https://googleapis.github.io/python-genai/genai.html#genai.client.Client
  response = client.models.generate_content(
      model=MODEL_NAME,
      contents=prompt,
      config=GenerateContentConfig(
          system_instruction=
            [
              'You are a helpful AI assistant.',
              'Your mission is to answer to the user question accuately.'
            ],
            tools=[google_search_tool],
            response_modalities=["TEXT"],
          )

    )
  print(response.text)

  return response.text

### Agent - Local unit test

In [ ]:
from vertexai import agent_engines

# Agent Define
agent = agent_engines.LangchainAgent(
    model=MODEL_NAME,
    tools=[
           get_exchange_rate,
           get_capital_return,
    ],

    agent_executor_kwargs={"return_intermediate_steps": False},
)

#### Test your agent locally


In [ ]:
# Agent local unit test before deploying.
agent.query(input="what's the capital city of south korea")

In [ ]:
# Agent local unit test before deploying.
agent.query(input="What's the exchange rate from US dollars to Korean Won currency on 2025-04-14 ?")

## End of Notebook